In [1]:
from backtesting import Backtest, Strategy
from backtesting.test import GOOG
from backtesting.lib import crossover
import seaborn as sns
#import backtesting
#backtesting.set_bokeh_output(notebook=False)
import pandas as pd
import numpy as np
import talib

In [2]:

def labelize_output(df: pd.DataFrame, target:list, wrt:str='Close', threshold:float=0.01, risk_reward_ratio:float=0.8) -> pd.DataFrame:
    
    df_ = df.copy()
    positive_criterion = 1 + threshold/risk_reward_ratio
    negative_criterion = 1 - threshold
     
    for column in target:
        conditions = [
            (df_[column] >= df_[wrt] * positive_criterion),
            (df_[column] <= df_[wrt] * negative_criterion)]
        choices = [2, 1]
        df_[f"target_{column}"] = np.select(conditions, choices, default=0)
        #df.drop(column,1,inplace=True)
    return df_

def TFs_to_predict(df:pd.DataFrame, apply_on:str, TimeFrame:str, howmanyTFs:int) -> pd.DataFrame:

    #apply_one = any('Open','Close','High','Low')
    for i in range(1, howmanyTFs+1):
        df[f"{TimeFrame}{apply_on}{i}"] = df["Close"].shift(-i)
    
    return df
    


In [3]:
df = pd.read_csv('test.csv')
df.drop(columns=['CloseAbovethreshold','HighAbovethreshold','LowAbovethreshold','OpenAbovethreshold','CloseBelowthreshold','HighBelowthreshold','LowBelowthreshold','OpenBelowthreshold','min_above','min_below'], inplace=True)
df['CloseTimestamp']=pd.to_datetime(df['CloseTimestamp']).dt.date

df

,Open,High,Low,Close,Volume,CloseTimestamp,TP,SL,signal
0,4261.48,4485.39,4200.74,4285.08,795.150377,2017-08-18,4413.6324,4220.80380,0
1,4285.08,4371.52,3938.77,4108.37,1199.888264,2017-08-19,4231.6211,4046.74445,0
2,4108.37,4184.69,3850.00,4139.98,381.309763,2017-08-20,4264.1794,4077.88030,0
3,4120.98,4211.08,4032.62,4086.29,467.083022,2017-08-21,4208.8787,4024.99565,0
4,4069.13,4119.62,3911.79,4016.00,691.743060,2017-08-22,4136.4800,3955.76000,0
...,...,...,...,...,...,...,...,...,...
1968,16831.85,17041.00,16679.00,16950.65,207401.284150,2023-01-07,17459.1695,16696.39025,1
1969,16950.31,16981.91,16908.00,16943.57,104526.568800,2023-01-08,17451.8771,16689.41645,1
1970,16943.83,17176.99,16911.00,17127.83,135155.896950,2023-01-09,17641.6649,16870.91255,0
1971,17127.83,17398.80,17104.66,17178.26,266211.527230,2023-01-10,17693.6078,16920.58610,0


# CROSSOVER STRATEGY

In [17]:
def optim_func(series):
    
    if series["# Trades"] < 10:
        return -1

    return series["Equity Final [$]"]/series["Exposure Time [%]"]



class RsiOscillator(Strategy):

    upper_bound = 70
    lower_bound = 30
    rsi_window = 14

    def init(self):
        # I = indicator
        self.rsi = self.I(talib.RSI, self.data.Close, self.rsi_window)
        self.ema7 = self.I(talib.EMA, self.data.Close, 7)
        self.ema30 = self.I(talib.EMA, self.data.Close, 30)
        self.ema60 = self.I(talib.EMA, self.data.Close, 60)
        self.ema200 = self.I(talib.EMA, self.data.Close, 200)

    def next(self):
        
        if crossover(self.rsi, self.upper_bound) or crossover(self.ema200, self.ema7) :
            self.position.close()
        elif crossover(self.lower_bound, self.rsi) or crossover(self.ema7, self.ema200):
            self.buy()
            

def generate_next_open(open):
    return pd.DataFrame(open).shift(1)

    
class SignalStrategy(Strategy):

    n_days=5

    def init(self):
        
        self.is_long = False
        self.days_passed = 0
        self.profit = 0
        self.entry_price = 0
        self.tp_perc = 1.03
        self.sl_perc = .985

    def next(self):
        current_signal = self.data.signal[-1]
        current_close = self.data.Close[-1]

        if current_signal == 1:
            if not self.position:
                    #self.buy()
                    self.buy(tp=self.tp_perc*current_close,  sl=self.sl_perc*current_close)
            
        if self.position:
            self.days_passed += 1

        if self.days_passed >= self.n_days:
                self.position.close()
                self.days_passed = 0

bt = Backtest(df, SignalStrategy, cash = 10_000, commission=0.01)

#bt = Backtest(df, RsiOscillator, cash = 10_000)
#stats = bt.optimize(
#    upper_bound = range(55,85,5),
#    lower_bound = range(10,45,5),
#    rsi_window = range(10, 30, 2),
#    maximize= "Sharpe Ratio"
#)

stats = bt.run()
bt.plot()   

C:\Users\ma.orlandi\AppData\Local\Temp\ipykernel_22424\3527121913.py:65: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(df, SignalStrategy, cash = 10_000, commission=0.01)
C:\Users\ma.orlandi\AppData\Local\Temp\ipykernel_22424\3527121913.py:65: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, SignalStrategy, cash = 10_000, commission=0.01)


Row(id='10939', ...)

In [6]:
stats

Start                                     0.0
End                                    1972.0
Duration                               1972.0
Exposure Time [%]                   30.663964
Equity Final [$]            1176982566.777301
Equity Peak [$]             1176982566.777301
Return [%]                    11769725.667773
Buy & Hold Return [%]               307.68317
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                   -0.908578
Avg. Drawdown [%]                   -0.268567
Max. Drawdown Duration                   21.0
Avg. Drawdown Duration                3.72973
# Trades                                433.0
Win Rate [%]                        99.076212
Best Trade [%]                       3.768675
Worst Trade [%]                     -0.500516
Avg. Trade [%]                    

In [ ]:
df[599:].groupby('signal').count()